In [9]:
from urllib import * #urlib2 has been moved to several modules
from urllib.request import urlopen
import tweepy
import json
import requests

In [2]:
credentials = {}
credentials['api_key'] = 'n2dTtVbZuJwji0auruBGVTBwT'
credentials['api_secret_key'] = 'U1Er7hNUFtOwmPIz6BCr6w1B3o9GlKhg6loffJVIK0cFAFdkYl'
credentials['access_token'] = '987093510116532231-tiHjc16Ws3vcMZpNUt0kuL7xj8iLSy1'
credentials['access_token_secret'] = 'ykZmkTz2TIeSajyWDKXvSSxAiFZvMFdD9ePUe0zn5fsna'

# Save the credentials object to file
with open("twitter_credentials.json", "w") as file:
    json.dump(credentials, file)

# Load Twitter API secrets from an external JSON file
api_key = credentials['api_key']
api_secret_key = credentials['api_secret_key']
access_token = credentials['access_token']
access_token_secret = credentials['access_token_secret']

# Twitter initialization
auth = tweepy.OAuthHandler(api_key, api_secret_key)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [3]:
def insert(myUser, db):
    #GET method
    payload = myUser
    url='http://localhost:8983/solr/'+db+'/update/json/docs'
    headers = {'content-type': 'application/json'}
    r = requests.post(url, data=json.dumps(payload), headers=headers)
    #print(r.json())

    #POST method
    url='http://localhost:8983/solr/'+db+'/update/json?commit=true%27'
    r = requests.get(url)
    #print(r.json())

In [30]:
def get_all_tweets(screen_name):
    # initialize a list to hold all the tweepy Tweets
    alltweets = []
    
    # make initial request for most recent tweets (200 is the maximum allowed count)
    new_tweets = api.user_timeline(screen_name = screen_name,count=200)
    
    # save most recent tweets
    alltweets.extend(new_tweets)
    
    # save the id of the oldest tweet less one
    oldest = alltweets[-1].id - 1
    
    # keep grabbing tweets until there are no tweets left to grab
    while len(new_tweets) > 0:
        print("getting tweets before %s" % (oldest))
        
        # all subsiquent requests use the max_id param to prevent duplicates
        new_tweets = api.user_timeline(screen_name = screen_name,count=200,max_id=oldest)
        
        # save most recent tweets
        alltweets.extend(new_tweets)
        
        # update the id of the oldest tweet less one
        oldest = alltweets[-1].id - 1
        
        print("...%s tweets downloaded so far" % (len(alltweets)))
        
    #Write into Solr   
    for tweet in alltweets:
        myTweet = {
            'id_str' : tweet.id_str,
            'user_id_str' : tweet.user.id_str,
            'created_at' : str(tweet.created_at),
            'user' : {
                'id_str' : tweet.user.id_str,
                'screen_name' : tweet.user.screen_name,
                'followers_count' : tweet.user.followers_count,
                'friends_count' : tweet.user.friends_count
            },
            'text' : tweet.text,
            'favorite_count' : tweet.favorite_count,
            'in_reply_to_screen_name' : tweet.in_reply_to_screen_name,
            'retweeted' : tweet.retweeted
        }
        insert(myTweet, 'Tweets')
    print('Finished inserting tweets from '+screen_name)

In [26]:
# Get the tweets between two dates
def query_1():
    print()

In [24]:
# Get the users who have 40 or more followers
def query_2():
    url='http://localhost:8983/solr/Users/select?q=followers_count%3A%20%5B40%20TO%20*%5D'
    r = requests.get(url)
    print(r.json()) #Tengo que imprimir bonito iterando el json

In [ ]:
if __name__ == '__main__':
    get_all_tweets("srcaballero99") #pass in the username of the account you want to insert
    get_all_tweets("Star_Lord987")
    get_all_tweets("Rush2Loot")
    #query_2()

getting tweets before 987094350722191359
...105 tweets downloaded so far
Finished inserting tweets from srcaballero99
getting tweets before 1233727576864120831
...392 tweets downloaded so far
getting tweets before 1186001783405137919
...580 tweets downloaded so far
getting tweets before 1149644054063304703
...753 tweets downloaded so far
getting tweets before 1064253782823968768
...937 tweets downloaded so far
getting tweets before 995449260232626175
...1128 tweets downloaded so far
getting tweets before 935080812978065408
...1326 tweets downloaded so far
getting tweets before 785558030926680064
...1345 tweets downloaded so far
getting tweets before 598532734101954559
...1345 tweets downloaded so far
